# **Tratamento de dados com PySpark!**
Aqui não estamos trabalhando com processamento em cluster, mas sim com o contexto PySpark em um single node.<br>
Então **cuidado** quando rodar ações pesadas como o collect(), por exemplo. 

## **Instalando JAVA_HOME e SPARK_HOME na máquina local - Anaconda + Windows 10**

Se você já tem o **JAVA** e **SPARK** instalados basta apontar para os diretórios específicos.<br>
Obs.: *As linhas comentadas abaixo são uma opção para baixar / criar os diretórios do JAVA e SPARK para usar com esse notebook*

In [41]:
#import time
#!curl -O https://enos.itcollege.ee/~jpoial/allalaadimised/jdk8/jdk-8u291-linux-x64.tar.gz
#time.sleep(5)
#!tar xf jdk-8u291-linux-x64.tar.gz
#time.sleep(5)
#!curl -O http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
#time.sleep(5)
#!tar xf spark-2.3.1-bin-hadoop2.7.tgz
#time.sleep(5)
#!pip install findspark


**Aqui vamos criar as HOMEs necessárias importando o OS.**<br> 
**Recomenddo rodar a célula abaixo mesmo se JAVA_HOME e SPARK_HOME já estiverem cadastrados nas variaveis de ambiente.**<br>
Obs.: Se você já tem o JAVA_HOME e SPARK_HOME configurados, basta ajustar os caminhos

In [40]:
import os
os.environ["JAVA_HOME"] = "jdk1.8.0_291" 
os.environ["SPARK_HOME"] = "spark-2.3.1-bin-hadoop2.7"


## **Instalando JAVA_HOME e SPARK_HOME no Google Colab**

Aqui vale lembrar que o Colab reseta após alu, então você terá que executar esse procedimento sempre que for trabalhar com o notebook.<br> 
Obs.: *No caso do colab o dataset tem que ser abixado sempre que o ambiente for resetado*

In [ ]:
#import time
#!apt-get update
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#time.sleep(5)
#!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
#time.sleep(5)
#!tar xf spark-2.3.1-bin-hadoop2.7.tgz
#time.sleep(5)
#!pip install -q findspark

In [ ]:
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

### **Criando o SparkContext e SparkSession**


In [38]:
import findspark
findspark.init()

In [ ]:

from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sc

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
#spark

<h1 style="background-color:  #3498db ; color: white;">
<b> PySpark DataFrames</h1>

###  **Baixando o Dataset e criando o Dataframe!**

Retire os comentários das linhas abaixo apenas se você não tem o arquivo.
Caso já tenha, deixe comentado. <br>
Esse é um dataset de **registro de crimes da cidade de Chicago** de **2001 até hoje** e tem quase **2GB**<br>
**Obs.:** *No caso do colab sempre lembrar se verificar se o arquivo ainda está disponível no ambiente.<br>
*Voce pode fazer isso usando o comando abaixo em uma célula tipo code*<br>
!ls -lh

**IMPORTANTE** -> Foi feita extensa busca por dados similares de uma cidade específica do **Oregon** mas infelizmente os registros não foram encontrados. **◬**

In [51]:
#!curl -O https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
#time.sleepp(5)    

#Se você estiver no colab (linux) vai renomear o arquivo usando o mv
#!mv 'rows.csv?accessType=DOWNLOAD' tretas_de_chicago.csv

#Se você estiver no windoes vai renomear o arquivo usando o move
#!move "rows.csv?accessType=DOWNLOAD" tretas_de_chicago.csv

C:\Users\f07699b\Documents\Residencia Dados\Treinamentos\Apache pyspark by example\scripts\rows.csv_accessType=DOWNLOAD
        1 arquivo(s) movido(s).


#### **read.csv()**
#### Aqui nós importamos os dados do CSV para um DataFrame PySpark

In [ ]:
from pyspark.sql.functions import to_timestamp, col, lit
rc = spark.read.csv('tretas_de_chicago.csv', header=True).withColumn('Date', to_timestamp(col('Date'),'MM/dd/yyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5, truncate = True)

#### **take()** 
#### Retorna  o conteudo de linhas do dataframe. O numero que passamos como argumento a funão vai representar o número de linhas coletadas rc.head() tem exatamente a mesma saída de rc.take(), lembrando que esse head() aqui do contecto Spark não é parecido com o head() do pandas. 

In [ ]:
rc.take(1) 

#### **collect()**
#### Coleta todos os dados do dataframe. Cuidado ao usar, pois pode cauisar um crash no driver node!
#### Se após rodar o collect() acontecer esse problema com seu *Jupter Notebook local* -->>  **Exception: Java gateway process exited before sending the driver its port number**, apague e descompacte novamente o diretório do **SPARK_HOME**
#### Como isso é apenas um LAB, essa opção é possível, então aproveite!

In [ ]:
#Vai testar?
#rc.collect()  

#### **show()**
#### Vai printar 3 linhas do dataset incuindo o header.  Esse sim é igualzinho a saída do .head() do Pandas.

In [ ]:
rc.show(3)

#### **count()**
#### Vai contar quantos registros temos no Dataframe

In [ ]:
rc.count()

## Schemas

O PySApark, com base nos dados define de forma automática o tipo de dados que está sendo importando. Porpem em situaçãoes de produção, é recomendado que o schema seja definido pelo usuário. Um exemplo são **datas** que na maioroia das vezes são importadas como **strings**<p>
Para trabalhar com schemas precisamos importar algumas coisas antes da biblioteca **pyspark.sql.types**<br>
**StructType**-->>Encapsula a estrutura do schema<br>
**StructField**-->> É usado na definição de cada campo<br>
**Type()**-->> Se refere ao tipo de campo. Pode ser **IntegerType**, **StringType**, **BooleanType**, etc... Acho que deu pra pegar a ideia.
    


In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DateType, FloatType, TimestampType, DoubleType
#Obs.: Não vamos usar tudo isso, mas é bom saber que temos várias opções

#### **printSchema()**
#### Usamos esse comando para ver o schema do dataframe. 
#### Baseado no output abaixo vemos que o campo "Date" é um timestamp, porém o campo "Updated On" que também é uma data, está como string. Precisamos ajustar isso.

In [ ]:
rc.printSchema()

#### **Para ajustar as os campos usamos a sintaxe abaixo**.
StrucType vai receber uma lista onde cada item é um StructField que recebe 3 argumentos<br>
#### 1 - Coluna
#### 2 - Filed Type
#### 3 - Se o campo pode ter nulos ou não (True | False)
Ex.:
**rc_schema = StructType([StructField('ID', StringType, True),StructField('Case Number', StringType, True)])**

Ps.: Devemos fazer a sequencia **StructField(Coluna, Type, True|False)** para TODAS as colunas. 
<p>
**DICA** <br>
Quando estiver trabalhando com o Schema de vários campos, a melhor coisa a fazer é tratar as colunas que são específicas primeiro.<br>
Depois criar uma tratativa padrão para as demais.<br>
No meu caso foi transformar elguns campos em Timestamp, Boolean e Double e depois todo o resto em string

In [ ]:
labels = rc.columns
labels
for coluna in range(len(labels)):
    if labels[coluna] == 'Date'or labels[coluna] == 'Updated On':
        labels[coluna] = (labels[coluna], TimestampType(), True)
    elif labels[coluna] == 'Arrest' or labels[coluna] == 'Domestic':
        labels[coluna] = (labels[coluna], BooleanType(), True)
    elif labels[coluna] == 'Year':
        labels[coluna] = (labels[coluna], IntegerType(),True)
    elif labels[coluna] =='Latitude' or labels[coluna] =='Longitude':
        labels[coluna] = (labels[coluna], DoubleType(), True)
    else: labels[coluna] = (labels[coluna], StringType(), True)
      
print(labels)


Aqui usamos uma **lambda function**, que vai passar por todos os items da lista labels e executar ação que adiciona os 3 valores 
de cada item daquela iteração na variavel screma. Os valores são: **Index 0** -> Coluna / **Index 1** -> field type / **Index 2** -> True ou False 

In [ ]:
schema = StructType([StructField (x[0],x[1],x[2]) for x in labels])
schema

#### **Carregando o Schema**
Aqui a gente pega o schema que foi criado e usa para carregar o CSV. Simples assim. 

In [ ]:
rc_schema = spark.read.csv('tretas_de_chicago.csv', schema = schema, header = True)
rc_schema.printSchema()

# **Trabalhando com Colunas**

#### **Selecionando colunas**

Podemos acessar as colunas em dataframe dem PySpark de duas maneiras:<p>
Por indexing  -> **df['Column_name']**<br>
Por função    -> **df.select(col('column_name'))** ou **df.select('column_name')**<p>
É importante lembrar que se o nome da coluna tiver espaços ou nomes reservados você **não vai conseguir acessar** usando o acesso via atributo

In [ ]:
rc.select(col('ID')).show(1)

In [ ]:
rc.select('ID','Date','Arrest').show(1)

In [ ]:
rc['ID', 'Date','Arrest'].show(1)

#### **Trabalhando com os headers**
Para retornar o header de um Dataframe em **PySpark** fazemos igual ao **Pandas**.<br>
E como sa saída é uma lista, podemos acessar essa lista via index ou usar outras ações aplicaveis a listas

In [ ]:
#Vaocê tira os comentários e testaa as saídas
rc.columns[0:3]
#rc.columns
#list(reversed(rc.columns))
#rc.columns[::-1]
#len(rc.columns)

#### **Adicionando novas colunas**

Pandas -> **df['coluna_nova'] = df['coluna_velha'] * 2**<br>
PySpark -> **df.withColumn('coluna_nova', 2 * df['coluna_velha']**

In [ ]:
rc = rc.withColumn('coluna_nova_2', rc['ID'] / 2)
rc.select('ID', 'coluna_nova_2').show(2) 

#### **Mudando os nomes das colunas**
Pandas  -> **df.rename(columns={'Nome_antigo':'Nome_novo'})**<br>
PySpark -> **df.withColumnRenamed('Nome_antigo','Nome_Novo')**

In [ ]:
rc = rc.withColumnRenamed('coluna_nova_2','IDx2')
rc.columns

#### **Removendo colunas**

PySpark -> **df = df.drop('column')**

In [ ]:
rc = rc.drop('IDx2')
rc.columns

# **Trabalhando Filtros e Linhas**

#### **filter()**
#### Diferente do Pandas, onde podemos filtrar direto na seleção da coluna, ex: df['coluna' > 50], em PySpark nós usamos a função filter()

In [ ]:
rc.filter(col('Date')  > '2017-11-11' ).show(2)

#### **distinct()**
#### Selecionando valores únicos em um dataframe
#### No pandas usamos **df['coluna'].unique()**, já aqui é um pouco diferente.

In [ ]:
rc.select('Arrest').distinct().show()

#### **count()**
#### Com ele contamos os valores selecionados

In [ ]:
prisoes = rc.filter(col('Arrest') == True).count() 
print(prisoes)


#### **orderBy()**
#### Usamos para fazer a ordenação do dataframe de acordo com a coluna selecionada

In [ ]:
rc.filter(col('District') != 'null').select(col('District')).distinct().orderBy(col('District')).show(1)

#### **groupBy()**
#### Usamos para agrupar valores de uma coluna específica e usar alguma agregação nesse resultado, como count(), sum(), entre outras.

In [ ]:
rc.groupBy('Arrest').count().show()

#### **union()**
#### **Concatenando Dataframes**
#### Seguindo as premissas já conhecidas de Python, Dataframes são imutaveis, deste modo não podemos fazer um append como fazemos com listas. Neste caso devemos concatenar os ataframes uns con os outros. 
#### **Critérios para concatenação**
Os Dataframes devem ter o mesmo numero de colunas<br>
Os Dataframes devem ter o mesmo schema
#### No Pandas nós usamnos **pd.concat(df,df2)**, no PySpark usamos:

In [ ]:
#Para esse exemplo usaremso uma parte menor do Dataframe
print(f"Para o Distrito 008 temos {rc.filter(col('District') == '008').count()} registros")
print(f"Para o Distrito 009 temos {rc.filter(col('District') == '009').count()} registros")
rc1 = rc.filter(col('District') == '008')
rc2 = rc.filter(col('District') == '009')
print(f"Apos usar o union() o total de registros do novo dataframe é {rc1.union(rc2).count()}, que representa a soma dos totais anteriores.")
     

# **Desafios**

#### **1) Quantos crimes resultaram em prisões?**

In [ ]:
'''
Primeiro temos que entender o time de dados que estamos manipulando e suas variações.
#Quantos valores possiveis temos para Arrest?
'''
rc.select(col('Arrest')).distinct().show()

In [ ]:
'''Qual o tipo de dado?'''
rc.printSchema()
#Arrest: string

In [ ]:
'''Agora fazemos a conta!'''
result = round((rc.filter(col('Arrest') == 'true').count() / rc.select(col('Arrest')).count()) * 100,2) 
print(f'A porcentagem de crimes que resultaram em prisões é de {result}%.')

#### **1) Quais o TOP 3 de locais mais perigosos de Chigaco?**

In [ ]:
'''
Primeiro vamos entender os dados. Temo um campo chamado Block (bairro)  que pode nos dar a resposta.
#Mas antes algumas premissas devem ser definidas: O tipo de "crime" terá algum peso na resposta ou
#apenas a quantidade de ocorrencias importa?
É sempre importante fazer esse tipo de pergunta, mas nesse caso específico, vamos de quantidade.
'''

#### Vamos entender as informações que temos usando o **groupBy()**:

In [ ]:
rc.groupBy('Block').count().show()

#### Agora que temos a lista dos blocks com a contagem dos crimes agregada precisamos penas odernar de acordo com nossa necessodade

In [ ]:
rc.groupBy('Block').count().orderBy('count', ascending = False).show(3)


#### **3) Quais o TOP 3 bairros com maior número de prisoes efetuadas?**

In [ ]:
'''
Essa foi fácil. Agora que sabemso como agrupar as informações com o groupBy() e fazer as agregações, basta filtar as informações.
Aqui vamos primeiro filtrar todos os registros onde o canpo Arrest é iguala  True. Depois só precisamos fazer as agregações.
'''

rc.filter(col('Arrest') == True).groupBy('Block').count().orderBy('count', ascending = False).show(3)

<h5 style="background-color:  #3498db ; color: white;">
<b> Essa é uma introdução básica de uso do DataFrame API do Pyspark.<br><br>
Existe muito mais conteúdo por ai, especialmente no website do atual mantenedor do Spark, a Databricks.<br> 
Para mais informações vá até https://academy.databricks.com <br><br>
Também recomendo os cursos do <b>Jonathan Fernandes</b>.<br> 
Ele tem ótimos cursos e foi de lá que tirei a ideia de usar esse dataset específico de Chicago transferindo o ambiente Spark para a máquina local usando Anaconda.<br>
</h5>

